Install the tweepy library

In [1]:
# !pip install tweepy
# !pip install mysqlclient

Import required libraries

In [2]:
import os
import re
import time

import tweepy as tw
import pandas as pd
from datetime import datetime, timedelta, timezone
import MySQLdb


# For sentiment analysis of tweets
from textblob import TextBlob

Create Tweeter Consumer Keys and Access Tokens 

In [3]:
consumer_key= 'NFIO7bIgwQpdCZ7p7rgewKXss'
consumer_secret= '2QegNrIpcGCW6nCWFTL5JEsrDCmYZpQHJ8SdIT7V0ql5cQgFwV'
access_token= '3049227557-36ln3qak9xbxHvgNos45cf3uScGxuUdaDAR4vqs'
access_token_secret= 'pZ4Y0p31graKrAYAy41mYztvIrYFzof7awiTDWQr2ui9u'

Initialize AuthHandler

In [4]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

Define search words and date

In [5]:
search_words = "#Covid"
date_since = "2018-11-16"

In [6]:
tweets = tw.Cursor(api.search, q=search_words, lang="en", since=date_since).items(5)

In [7]:
tweets

Iterate through tweets using for loop

In [8]:
from datetime import datetime, timedelta
from email.utils import parsedate_tz

s='Mon Dec 07 17:03:03 +0000 2020'
def to_datetime(datestring):
    time_tuple = parsedate_tz(datestring.strip())
    dt = datetime(*time_tuple[:6])
    return dt - timedelta(seconds=time_tuple[-1])

print(to_datetime(s))
print(to_datetime(s).time().strftime("%H:%M:%S"))

x=datetime.strptime('12/31/13 00:12', "%m/%j/%y %H:%M").time().strftime("%H:%M:%S")

print(x)

2020-12-07 17:03:03
17:03:03
00:12:00


Retrieve tweets as a collection of list

In [9]:
[tweet.text for tweet in tweets]

['#UK dreams for getting rid of #covid-19! https://t.co/NaHURX2nVc',
 'RT @VABVOX: If you were looking for the worst people in Congress, they are:\n▪Marsha Blackburn \n▪Ted Cruz\n▪Ron Johnson\n▪Mike Lee\n▪Rand Paul…',
 'RT @NWT_CPHO: THREAD 1/2 – The #CovidAlert App is LIVE in #NWT. By exchanging anonymous Bluetooth codes with nearby phones, the app can tel…',
 "Latest news on #RohitSharma\n\n #Sydney #Covid cluster unlikely to affect #Rohit's participation \n\n@CricCrazyJohns… https://t.co/rXvTYbNmdm",
 'Look at this idiot, trying to take credit for #Covid vaccine that be initially called a hoax https://t.co/kisQR0ry7A']

Search Tweets without Retweets

In [10]:
new_search = search_words + " -filter:retweets"

In [11]:
new_search

'#Covid -filter:retweets'

In [12]:
tweets = tw.Cursor(api.search, q=new_search, lang="en",since=date_since).items(5)

In [13]:
[tweet.text for tweet in tweets]

["Latest news on #RohitSharma\n\n #Sydney #Covid cluster unlikely to affect #Rohit's participation \n\n@CricCrazyJohns… https://t.co/rXvTYbNmdm",
 'Look at this idiot, trying to take credit for #Covid vaccine that be initially called a hoax https://t.co/kisQR0ry7A',
 'Why are we shipping money to Pakistan when Americans are starving because of the #COVID shutdowns? #stimulus',
 '@WhiteHouse We should have gotten a #COVID19 vaccine sooner but \n@realDonaldTrump downplayed the dangers of #COVID… https://t.co/7nVYadq0kq',
 'Markets fall as more nations join #UK travel ban over #Covid variant; lorry drivers stranded, supermarkets warn of… https://t.co/CjmEs9hRoq']

Get username and location

In [14]:
tweets = tw.Cursor(api.search,q=new_search, lang="en", since=date_since).items(5)

In [15]:
users_locs = [[tweet.user.screen_name,tweet.user.location,tweet.text] for tweet in tweets]

In [16]:
users_locs

[['Being_SAi143',
  'мuмвɑᎥ',
  "Latest news on #RohitSharma\n\n #Sydney #Covid cluster unlikely to affect #Rohit's participation \n\n@CricCrazyJohns… https://t.co/rXvTYbNmdm"],
 ['dangainor',
  'Reston, VA.',
  'Why are we shipping money to Pakistan when Americans are starving because of the #COVID shutdowns? #stimulus'],
 ['soloyochapin',
  'Chicago, IL',
  '@WhiteHouse We should have gotten a #COVID19 vaccine sooner but \n@realDonaldTrump downplayed the dangers of #COVID… https://t.co/7nVYadq0kq'],
 ['Diplomacy140',
  'London, England',
  'Markets fall as more nations join #UK travel ban over #Covid variant; lorry drivers stranded, supermarkets warn of… https://t.co/CjmEs9hRoq'],
 ['The_Truth_II',
  'Praia da Luz,  Portugal ',
  'How long are we prepared to live like this? #covid #lockdown #tiersystem #masks #housearrest #roadblocks… https://t.co/phIYFc9jvw']]

Create a Pandas Dataframe From A List of Tweet Data

In [17]:
tweet_df = pd.DataFrame(data=users_locs, columns=['user', "location","tweet"])

In [18]:
tweet_df

,user,location,tweet
0,Being_SAi143,мuмвɑᎥ,Latest news on #RohitSharma\n\n #Sydney #Covid...
1,dangainor,"Reston, VA.",Why are we shipping money to Pakistan when Ame...
2,soloyochapin,"Chicago, IL",@WhiteHouse We should have gotten a #COVID19 v...
3,Diplomacy140,"London, England",Markets fall as more nations join #UK travel b...
4,The_Truth_II,"Praia da Luz, Portugal",How long are we prepared to live like this? #c...


## Working With The Twitter Streaming API

With Streaming data we need to do the following:
1. Create database connection
2. Create a persistent connection to the Twitter API, and read each connection incrementally.
3. Process tweets quickly, store them in the database, and don’t let your program get backed up.
4. Handle errors and other issues properly.

Connect to MySql Database

In [19]:
#                           (server,MySQL username,MySQL pass, Database name).
connection = MySQLdb.connect("localhost","root","root","social_analytic")
cur = connection.cursor()

StreamListener Class

In [ ]:
from tweepy import Stream
from tweepy.streaming import StreamListener
import tweepy
import time
import json
import sys


search_val="covid"

class listener(StreamListener):

    def on_data(self, data):
       
        try:
#             tweet_count=0
            
            all_data = json.loads(data)
            
#             search_val="covid"
            tweet_id=all_data["id"]
            created_at=to_datetime(all_data["created_at"])
            created_at_date=created_at.date()
            created_at_hour=created_at.time().strftime("%H")
            created_at_min=created_at.time().strftime("%H:%M")
            text = all_data["text"]
            source = all_data["source"]
#             local_created_at=created_at.str.replace(tzinfo=timezone.utc).astimezone(tz=None)
            local_created_at=all_data["created_at"]
            
            in_reply_to_status_id=all_data['in_reply_to_status_id_str']
            in_reply_to_user_id=all_data['in_reply_to_user_id_str']
            in_reply_to_screen_name=all_data['in_reply_to_screen_name']
            username=all_data['user']['name']
            user_screen_name=all_data['user']['screen_name']
            user_location=all_data['user']['location']
            user_url=all_data['user']['url']
            user_description=all_data['user']['description']
            user_verified_str=all_data['user']['verified']
            user_verified=0
            if user_verified_str==True:
                user_verified=1
            user_followers_count=all_data['user']['followers_count']
            user_friends_count=all_data['user']['friends_count']
            user_listed_count=all_data['user']['listed_count']
            user_favourites_count=all_data['user']['favourites_count']
            user_created_at=to_datetime(all_data['user']['created_at'])
            user_created_at_date=user_created_at.date()
            user_id=all_data['user']['id']
            user_profile_image_url_https=all_data['user']['profile_image_url_https']
            
#             coordinates_lat=all_data['coordinates'][0]
#             coordinates_lon=all_data['coordinates'][1]
            if all_data['coordinates']==True:
                coordinates_lat=all_data['coordinates'][0]
                coordinates_lon=all_data['coordinates'][1]
            else:
                coordinates_lat=None
                coordinates_lon=None
            if all_data['place']==True:
                place_country=all_data['place_country']
                place_country_code=all_data['place_country_code']
                place_full_name=all_data['place_full_name']
                place_id=all_data['place_id']
                place_type=all_data['place_type']
            else:
                place_country=None
                place_country_code=None
                place_full_name=None
                place_id=None
                place_type=None
                
            retweeted_status=None
            
            if all_data['is_quote_status']==True:
                quoted_status_id=all_data['quoted_status_id']
                quoted_status=all_data['quoted_status']
#                 retweeted_status=all_data['retweeted_status']
                quote_count=0
            else:
                quoted_status_id=None
                quote_count=0
                quoted_status=None
#                 retweeted_status=None
            try:
                reply_count=all_data['reply_count']
            except:
                reply_count=0

            retweet_count=all_data['retweet_count']
            favorite_count=all_data['favorite_count']
            retweeted_txt=all_data['retweeted']
            entities=str(all_data['entities'])
            retweeted=0
            if retweeted_txt==True:
                retweeted=1
#             feeds_link=all_data['https://twitter.com/_/status/'+id]
            feeds_link=None
            lang=all_data['lang']
            feeds_image=None
            feeds_video=None
            # clean text
            text_preprocess = lambda x: re.compile('\#').sub('', re.compile('RT @').sub('@', x).strip())
            clean_text=text_preprocess(text)
            clean_text=re.sub('@[^\s]+','',clean_text)
            clean_text=re.sub(r"http\S+", "", clean_text)
            # end clean text
            text_sentiment_polarity=TextBlob(clean_text).sentiment[0]
            text_sentiment_subjectivity=TextBlob(clean_text).sentiment[1]


            cur.execute("INSERT INTO tweet (search_val,  created_at, created_at_date,created_at_hour,created_at_min, tweet_id,  text, source,in_reply_to_status_id, in_reply_to_user_id,  in_reply_to_screen_name,  user_name,  user_screen_name,  user_location, user_url,  user_description, user_verified, user_followers_count,  user_friends_count, user_listed_count,  user_favourites_count,  user_created_at,user_created_at_date, user_id,  user_profile_image_url_https,  coordinates_lat,  coordinates_lon, place_country,  place_country_code,  place_full_name, place_id,  place_type,  quoted_status_id,  quoted_status, retweeted_status,  quote_count,  reply_count,  retweet_count,  favorite_count, retweeted,  entities,  lang, feeds_link,  feeds_video, feeds_image,clean_text,text_sentiment_polarity,text_sentiment_subjectivity) VALUES (%s, %s,%s, %s,%s, %s,%s,%s,%s, %s, %s, %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", 
                        (search_val,created_at,created_at_date,created_at_hour,created_at_min,tweet_id,text,source,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,username,user_screen_name,user_location,user_url,user_description,user_verified,user_followers_count,user_friends_count,user_listed_count,user_favourites_count,user_created_at,user_created_at_date,user_id,user_profile_image_url_https,coordinates_lat,coordinates_lon,place_country,place_country_code,place_full_name,place_id,place_type,quoted_status_id,quoted_status,retweeted_status,quote_count,reply_count,retweet_count,favorite_count,retweeted,entities,lang,feeds_link,feeds_video,feeds_image,clean_text,text_sentiment_polarity,text_sentiment_subjectivity))

            connection.commit()
            print("Fetched tweets ...........\n",text)
            print(created_at_hour,created_at_min)
#             tweet_count +=1

            return True

# Catch all the errors from Database or Twiteer
        except tweepy.error.TweepError:
            print("Error fetching the data")
            
        except UnicodeEncodeError:
            pass
        
        except MySQLdb.DataError as e:
            print("DataError")
            print(e)

        except MySQLdb.InternalError as e:
            print("InternalError")
            print(e)

        except MySQLdb.IntegrityError as e:
            print("IntegrityError")
            print(e)

        except MySQLdb.OperationalError as e:
            print("OperationalError")
            print(e)

        except MySQLdb.NotSupportedError as e:
            print("NotSupportedError")
            print(e)

        except MySQLdb.ProgrammingError as e:
            print("ProgrammingError")
            print(e)
        except:
            print ("Unexpected general error: ", sys.exc_info()[0])
            raise
        finally:
            print("  ")
            #             cur.close()
            #             connection.close()
            

    def on_error(self, status):
        print (status)